In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 7.5 MB/s 
     |████████████████████████████████| 88 kB 8.7 MB/s 
     |████████████████████████████████| 5.2 MB 43.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 5.5 MB/s 
     |████████████████████████████████| 662 kB 70.9 MB/s 
     |████████████████████████████████| 5.8 MB 55.7 MB/s 
     |████████████████████████████████| 1.3 MB 51.2 MB/s 
     |████████████████████████████████| 182 kB 51.3 MB/s 
     |████████████████████████████████| 7.6 MB 47.2 MB/s 
     |████████████████████████████████| 1.1 MB 45.4 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700811 sha256=902e27f0aa9cf927257e70b6d4af0bccd016f9a578ad749868b4fbed834edf0f
  Stored in directory: /root/.cache/

In [4]:
from bertopic import BERTopic
BERT_model_FINAL = BERTopic.load("/content/drive/MyDrive/collectionandqueries/20KDocs/qrels_BERT_model_FINAL_preprocessed")

In [5]:
from math import log, sqrt
ALPHA = 1
BETA = 0.75
GAMMA = 0.15

def generateInvertedIndex(invertedIndex, doc):
        line = doc
        text = line.strip().split()
        for word in text:
            if word not in invertedIndex.keys():
                invertedIndex[word] = 1
            else:
                invertedIndex[word] += 1

def queryFrequency(query):
    queryFreq = {}
    for term in query.split():
        if term in queryFreq.keys():
            queryFreq[term] += 1
        else:
            queryFreq[term] = 1
    return queryFreq

def calculateDocsCount(doc, docIndex):
    line = doc
    text = line.strip().split()
    for term in text:
      if term in docIndex.keys():
        docIndex[term] += 1
      else:
        docIndex[term] = 1

def findDocs(all_docs):
    relIndex = {}
    k = len(all_docs)
    for i in range(0, k):
      doc = all_docs[i]
      calculateDocsCount(doc, relIndex)
    return relIndex

def findRelDocMagnitude(docIndex):
    mag = 0
    for term in docIndex:
        mag += float(docIndex[term]**2)
        mag = float(sqrt(mag))
    return mag

def findRocchioScore(term, queryFreq, relDocMag, relIndex):
    if(term not in queryFreq.keys()):
      queryFreq[term] = 0
    if(term not in relIndex.keys()):
      relIndex[term] = 0  
    rocchioScore = ALPHA * queryFreq[term] + (BETA/relDocMag) * relIndex[term] 
    return rocchioScore


def findNewQuery(query, all_docs):
    invertedIndex = {}
    for doc in all_docs:
      generateInvertedIndex(invertedIndex, doc) 
    queryFreq = queryFrequency(query)
    relIndex = findDocs(all_docs)
    relDocMag = findRelDocMagnitude(relIndex)
    updatedQuery = {}
    newQuery = query
    for term in invertedIndex.keys():
        updatedQuery[term] = findRocchioScore(term, queryFreq, relDocMag, relIndex)
    sortedUpdatedQuery = sorted(updatedQuery.items(), key=lambda x:x[1], reverse=True)
    if len(sortedUpdatedQuery)<5:
        loopRange = len(sortedUpdatedQuery)
    else:
        loopRange = 5
    for i in range(loopRange):
        term,frequency = sortedUpdatedQuery[i]
        if term not in query:
            newQuery +=  " "
            newQuery +=  term
    return newQuery

In [15]:
cur_query = "what drugs have been active against SARS-CoV or SARS-CoV-2 in animal studies?what types of rapid testing for Covid-19 have been developed?"

In [16]:
topics_res, similarity = BERT_model_FINAL.find_topics(cur_query, top_n=2) 
for j in range(0, 2):
  all_topic_docs = BERT_model_FINAL.get_representative_docs(topics_res[j])
  newQuery = findNewQuery(cur_query, all_topic_docs)
  print("newQuery : " + newQuery)

newQuery : what drugs have been active against SARS-CoV or SARS-CoV-2 in animal studies?what types of rapid testing for Covid-19 have been developed? hiv infection means antibody
newQuery : what drugs have been active against SARS-CoV or SARS-CoV-2 in animal studies?what types of rapid testing for Covid-19 have been developed? hepatitis virus incubation period get


In [ ]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
"""
This examples show how to train a Cross-Encoder for the MS Marco dataset (https://github.com/microsoft/MSMARCO-Passage-Ranking).
The query and the passage are passed simoultanously to a Transformer network. The network then returns
a score between 0 and 1 how relevant the passage is for a given query.
The resulting Cross-Encoder can then be used for passage re-ranking: You retrieve for example 100 passages
for a given query, for example with ElasticSearch, and pass the query+retrieved_passage to the CrossEncoder
for scoring. You sort the results then according to the output of the CrossEncoder.
This gives a significant boost compared to out-of-the-box ElasticSearch / BM25 ranking.
Running this script:
python train_cross-encoder.py
"""
from torch.utils.data import DataLoader
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CERerankingEvaluator
from sentence_transformers import InputExample
from datetime import datetime
import gzip
import os
import tarfile


#First, we define the transformer model we want to fine-tune
model_name = 'distilroberta-base'
train_batch_size = 32
num_epochs = 1
model_save_path = 'output/training_ms-marco_cross-encoder-'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


# We train the network with as a binary label task
# Given [query, passage] is the label 0 = irrelevant or 1 = relevant?
# We use a positive-to-negative ratio: For 1 positive sample (label 1) we include 4 negative samples (label 0)
# in our training setup. For the negative samples, we use the triplets provided by MS Marco that
# specify (query, positive sample, negative sample).
pos_neg_ration = 4

# Maximal number of training samples we want to use
max_train_samples = 8

#We set num_labels=1, which predicts a continous score between 0 and 1
model = CrossEncoder(model_name, num_labels=1, max_length=512)


### Now we read the MS Marco dataset
data_folder = '/content/drive/MyDrive/collectionandqueries/'
os.makedirs(data_folder, exist_ok=True)


#### Read the corpus files, that contain all the passages. Store them in the corpus dict
corpus = {}
final_full_docs = []
collection_filepath = os.path.join(data_folder, 'collection.tsv')
if not os.path.exists(collection_filepath):
    print('collection.tsv not found')

with open(collection_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        pid, passage = line.strip().split("\t")
        corpus[pid] = passage
        final_full_docs.append(passage)
print('len(corpus.keys() : ' + str(len(corpus.keys())))

from rank_bm25 import BM25Okapi

print('len(final_full_docs) : ' + str(len(final_full_docs)))

tokenized_corpus = [doc.split(" ") for doc in final_full_docs]
bm25 = BM25Okapi(tokenized_corpus)

### Read the train queries, store in queries dict
queries = {}
queries_filepath = os.path.join(data_folder, 'queries.train.tsv')
if not os.path.exists(queries_filepath):
    print('queries.train.tsv not found')

with open(queries_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        queries[qid] = query

print('len(queries.keys() : ' + str(len(queries.keys())))

### Now we create our training & dev data
train_samples = []
dev_samples = {}

# We use 200 random queries from the train set for evaluation during training
# Each query has at least one relevant and up to 200 irrelevant (negative) passages
num_dev_queries = 2
num_max_dev_negatives = 2

# msmarco-qidpidtriples.rnd-shuf.train-eval.tsv.gz and msmarco-qidpidtriples.rnd-shuf.train.tsv.gz is a randomly
# shuffled version of qidpidtriples.train.full.2.tsv.gz from the MS Marco website
# We extracted in the train-eval split 500 random queries that can be used for evaluation during training
train_eval_filepath = os.path.join(data_folder, 'msmarco-qidpidtriples.rnd-shuf.train-eval.tsv')
if not os.path.exists(train_eval_filepath):
    print('msmarco-qidpidtriples.rnd-shuf.train-eval.tsv not found')

with open(train_eval_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, pos_id, neg_id = line.strip().split()

        if qid not in dev_samples and len(dev_samples) < num_dev_queries:
            dev_samples[qid] = {'query': queries[qid], 'positive': set(), 'negative': set()}

        if qid in dev_samples:
            dev_samples[qid]['positive'].add(corpus[pos_id])

            if len(dev_samples[qid]['negative']) < num_max_dev_negatives:
                dev_samples[qid]['negative'].add(corpus[neg_id])

print('len(dev_samples.keys()) : ' + str(len(dev_samples.keys())))

# Read our training file
train_filepath = os.path.join(data_folder, 'qidpidtriples-subset.train-200K.tsv')
if not os.path.exists(train_filepath):
    print('qidpidtriples-subset.train-200K.tsv not found')

cnt = 0
with open(train_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, pos_id, neg_id = line.strip().split()

        if qid in dev_samples:
            continue

        query = queries[qid]
        if (cnt % (pos_neg_ration+1)) == 0:
            passage = corpus[pos_id]
            label = 1
            tokenized_query = query.split(" ")
            passages = bm25.get_top_n(tokenized_query, final_full_docs, n=3)
            train_samples.append(InputExample(texts=[query, passage], label=label))
        else:
            passage = corpus[neg_id]
            label = 0

        train_samples.append(InputExample(texts=[query, passage], label=label))
        cnt += 1

        if cnt >= max_train_samples:
            break
print('len(train_samples) : ' + str(len(train_samples)))

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight'

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

len(corpus.keys() : 8841823
len(final_full_docs) : 8841823


In [ ]:
pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bertopic import BERTopic
BERT_model_FINAL = BERTopic.load("/content/drive/MyDrive/collectionandqueries/20KDocs/qrels_BERT_model_FINAL_preprocessed")

In [ ]:
from math import log, sqrt
ALPHA = 1
BETA = 0.75
GAMMA = 0.15

def generateInvertedIndex(invertedIndex, doc):
        line = doc
        text = line.strip().split()
        for word in text:
            if word not in invertedIndex.keys():
                invertedIndex[word] = 1
            else:
                invertedIndex[word] += 1

def queryFrequency(query):
    queryFreq = {}
    for term in query.split():
        if term in queryFreq.keys():
            queryFreq[term] += 1
        else:
            queryFreq[term] = 1
    return queryFreq

def calculateDocsCount(doc, docIndex):
    line = doc
    text = line.strip().split()
    for term in text:
      if term in docIndex.keys():
        docIndex[term] += 1
      else:
        docIndex[term] = 1

def findDocs(all_docs):
    relIndex = {}
    k = len(all_docs)
    for i in range(0, k):
      doc = all_docs[i]
      calculateDocsCount(doc, relIndex)
    return relIndex

def findRelDocMagnitude(docIndex):
    mag = 0
    for term in docIndex:
        mag += float(docIndex[term]**2)
        mag = float(sqrt(mag))
    return mag

def findRocchioScore(term, queryFreq, relDocMag, relIndex):
    if(term not in queryFreq.keys()):
      queryFreq[term] = 0
    if(term not in relIndex.keys()):
      relIndex[term] = 0  
    rocchioScore = ALPHA * queryFreq[term] + (BETA/relDocMag) * relIndex[term] 
    return rocchioScore


def findNewQuery(query, all_docs):
    invertedIndex = {}
    for doc in all_docs:
      generateInvertedIndex(invertedIndex, doc) 
    queryFreq = queryFrequency(query)
    relIndex = findDocs(all_docs)
    relDocMag = findRelDocMagnitude(relIndex)
    updatedQuery = {}
    newQuery = query
    for term in invertedIndex.keys():
        updatedQuery[term] = findRocchioScore(term, queryFreq, relDocMag, relIndex)
    sortedUpdatedQuery = sorted(updatedQuery.items(), key=lambda x:x[1], reverse=True)
    if len(sortedUpdatedQuery)<5:
        loopRange = len(sortedUpdatedQuery)
    else:
        loopRange = 5
    for i in range(loopRange):
        term,frequency = sortedUpdatedQuery[i]
        if term not in query:
            newQuery +=  " "
            newQuery +=  term
    return newQuery

In [ ]:
query = "salary health"
topics_res, similarity = BERT_model_FINAL.find_topics(query, top_n=3)
all_topic_docs = BERT_model_FINAL.get_representative_docs(topics_res[0])
newQuery = findNewQuery(query, all_topic_docs)
print('newQuery : ' + newQuery)

all_topic_docs = BERT_model_FINAL.get_representative_docs(topics_res[1])
newQuery = findNewQuery(query, all_topic_docs)
print('newQuery : ' + newQuery)


all_topic_docs = BERT_model_FINAL.get_representative_docs(topics_res[2])
newQuery = findNewQuery(query, all_topic_docs)
print('newQuery : ' + newQuery)

newQuery : salary health nurse average illinois support
newQuery : salary health medicare plan part advantage c
newQuery : salary health hsa money deduct funds


In [ ]:
query = ""
topics_res, similarity = BERT_model_FINAL.find_topics(query, top_n=3)
all_topic_docs = BERT_model_FINAL.get_representative_docs(topics_res[0])
newQuery = findNewQuery(query, all_topic_docs)
print('newQuery : ' + newQuery)

In [ ]:
query = ""
topics_res, similarity = BERT_model_FINAL.find_topics(query, top_n=3)
all_topic_docs = BERT_model_FINAL.get_representative_docs(topics_res[0])
newQuery = findNewQuery(query, all_topic_docs)
print('newQuery : ' + newQuery)

In [ ]:
query = ""
topics_res, similarity = BERT_model_FINAL.find_topics(query, top_n=3)
all_topic_docs = BERT_model_FINAL.get_representative_docs(topics_res[0])
newQuery = findNewQuery(query, all_topic_docs)
print('newQuery : ' + newQuery)

In [ ]:
"""
This examples show how to train a Cross-Encoder for the MS Marco dataset (https://github.com/microsoft/MSMARCO-Passage-Ranking).
The query and the passage are passed simoultanously to a Transformer network. The network then returns
a score between 0 and 1 how relevant the passage is for a given query.
The resulting Cross-Encoder can then be used for passage re-ranking: You retrieve for example 100 passages
for a given query, for example with ElasticSearch, and pass the query+retrieved_passage to the CrossEncoder
for scoring. You sort the results then according to the output of the CrossEncoder.
This gives a significant boost compared to out-of-the-box ElasticSearch / BM25 ranking.
Running this script:
python train_cross-encoder.py
"""
import pickle
from torch.utils.data import DataLoader
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CERerankingEvaluator
from sentence_transformers import InputExample
from datetime import datetime
import gzip
import os
import tarfile

from rank_bm25 import BM25Okapi


def generateParaphrasedQueries(query):
  k = 3
  queries = []
  for i in range(0, 3): 
    topics_res, similarity = BERT_model_FINAL.find_topics(query, top_n=3)
    all_topic_docs = BERT_model_FINAL.get_representative_docs(topics_res[0])
    newQuery = findNewQuery(query, all_topic_docs)
    #print('newQuery : ' + newQuery)
    queries.append(newQuery)
  return queries

#First, we define the transformer model we want to fine-tune
model_name = 'distilroberta-base'
train_batch_size = 32
num_epochs = 1
model_save_path = 'output/training_ms-marco_cross-encoder-'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


# We train the network with as a binary label task
# Given [query, passage] is the label 0 = irrelevant or 1 = relevant?
# We use a positive-to-negative ratio: For 1 positive sample (label 1) we include 4 negative samples (label 0)
# in our training setup. For the negative samples, we use the triplets provided by MS Marco that
# specify (query, positive sample, negative sample).
pos_neg_ration = 4

# Maximal number of training samples we want to use
max_train_samples = 8

#We set num_labels=1, which predicts a continous score between 0 and 1
model = CrossEncoder(model_name, num_labels=1, max_length=512)


### Now we read the MS Marco dataset
data_folder = '/content/drive/MyDrive/collectionandqueries/'
os.makedirs(data_folder, exist_ok=True)


#### Read the corpus files, that contain all the passages. Store them in the corpus dict
corpus = {}
final_full_docs = []
collection_filepath = os.path.join(data_folder, 'collection.tsv')
if not os.path.exists(collection_filepath):
    print('collection.tsv not found')

with open(collection_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        pid, passage = line.strip().split("\t")
        corpus[pid] = passage
        final_full_docs.append(passage)
print('len(corpus.keys() : ' + str(len(corpus.keys())))
print('len(final_full_docs) : ' + str(len(final_full_docs)))

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 

len(corpus.keys() : 8841823
len(final_full_docs) : 8841823


In [ ]:
### Read the train queries, store in queries dict
queries = {}
queries_filepath = os.path.join(data_folder, 'queries.train.tsv')
if not os.path.exists(queries_filepath):
    print('queries.train.tsv not found')

with open(queries_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        queries[qid] = query

print('len(queries.keys() : ' + str(len(queries.keys())))

### Now we create our training & dev data
train_samples = []
dev_samples = {}

# We use 200 random queries from the train set for evaluation during training
# Each query has at least one relevant and up to 200 irrelevant (negative) passages
num_dev_queries = 2
num_max_dev_negatives = 2

# msmarco-qidpidtriples.rnd-shuf.train-eval.tsv.gz and msmarco-qidpidtriples.rnd-shuf.train.tsv.gz is a randomly
# shuffled version of qidpidtriples.train.full.2.tsv.gz from the MS Marco website
# We extracted in the train-eval split 500 random queries that can be used for evaluation during training
train_eval_filepath = os.path.join(data_folder, 'msmarco-qidpidtriples.rnd-shuf.train-eval.tsv')
if not os.path.exists(train_eval_filepath):
    print('msmarco-qidpidtriples.rnd-shuf.train-eval.tsv not found')

with open(train_eval_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, pos_id, neg_id = line.strip().split()

        if qid not in dev_samples and len(dev_samples) < num_dev_queries:
            dev_samples[qid] = {'query': queries[qid], 'positive': set(), 'negative': set()}

        if qid in dev_samples:
            dev_samples[qid]['positive'].add(corpus[pos_id])

            if len(dev_samples[qid]['negative']) < num_max_dev_negatives:
                dev_samples[qid]['negative'].add(corpus[neg_id])

print('len(dev_samples.keys()) : ' + str(len(dev_samples.keys())))

# Read our training file
train_filepath = os.path.join(data_folder, 'qidpidtriples-subset.train-200K.tsv')
if not os.path.exists(train_filepath):
    print('qidpidtriples-subset.train-200K.tsv not found')

cnt = 0
with open(train_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, pos_id, neg_id = line.strip().split()

        if qid in dev_samples:
            continue

        query = queries[qid]
        if (cnt % (pos_neg_ration+1)) == 0:
            passage = corpus[pos_id]
            label = 1
            paraphrased_queries = generateParaphrasedQueries(query)
            for new_query in paraphrased_queries:
              tokenized_query = new_query.split(" ")
              passages = bm25.get_top_n(tokenized_query, final_full_docs, n=3)
              train_samples.append(InputExample(texts=[new_query, passage], label=label))
        else:
            passage = corpus[neg_id]
            label = 0

        train_samples.append(InputExample(texts=[query, passage], label=label))
        cnt += 1

        if cnt >= max_train_samples:
            break
print('len(train_samples) : ' + str(len(train_samples)))

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.bias

len(corpus.keys() : 8841823
len(final_full_docs) : 8841823


In [ ]:
tokenized_corpus = [doc.split(" ") for doc in final_full_docs]
bm25 = BM25Okapi(tokenized_corpus)
with open("/content/drive/MyDrive/collectionandqueries/BM25", 'wb') as bm25File:
  pickle.dump(bm25)

In [ ]:
query = "define extreme"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)
# array([0. , 0.93729472, 0. ])
bm25.get_top_n(tokenized_query, final_full_docs, n=3)
# ['It is quite windy in London']

In [ ]:
for query in 
query = "define extreme"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)
# array([0. , 0.93729472, 0. ])
bm25.get_top_n(tokenized_query, final_full_docs, n=20)
# ['It is quite windy in London']